Import everything

In [1]:
import tree_maker
import yaml
import pandas as pd
import time
import logging


Preliminiary info

In [2]:
study_name = "example_tunescan"

Load job tree

In [3]:
# Start of the script
print("Analysis of output simulation files started")
start = time.time()

# Load Data

fix = "/scans/" + study_name
root = tree_maker.tree_from_json(fix[1:] + "/tree_maker_" + study_name + ".json")
# Add suffix to the root node path to handle scans that are not in the root directory
root.add_suffix(suffix=fix)

Analysis of output simulation files started


Browse simulations folder and extract relevant observables

In [4]:
l_problematic_sim = []
l_df_to_merge = []
for node in root.generation(1):
    with open(f"{node.get_abs_path()}/config.yaml", "r") as fid:
        config_parent = yaml.safe_load(fid)
    for node_child in node.children:
        with open(f"{node_child.get_abs_path()}/config.yaml", "r") as fid:
            config_child = yaml.safe_load(fid)
        try:
            particle = pd.read_parquet(
                f"{node_child.get_abs_path()}/{config_child['config_simulation']['particle_file']}"
            )
            df_sim = pd.read_parquet(f"{node_child.get_abs_path()}/output_particles.parquet")

        except Exception as e:
            print(e)
            l_problematic_sim.append(node_child.get_abs_path())
            continue

        # Register paths and names of the nodes
        df_sim["path base collider"] = f"{node.get_abs_path()}"
        df_sim["name base collider"] = f"{node.name}"
        df_sim["path simulation"] = f"{node_child.get_abs_path()}"
        df_sim["name simulation"] = f"{node_child.name}"

        # Get node parameters as dictionnaries for parameter assignation
        dic_child_collider = node_child.parameters["config_collider"]
        dic_child_simulation = node_child.parameters["config_simulation"]
        try:
            dic_parent_collider = node.parameters["config_mad"]
        except:
            print("No parent collider could be loaded")
        dic_parent_particles = node.parameters["config_particles"]

        # Get which beam is being tracked
        df_sim["beam"] = dic_child_simulation["beam"]

        # Get scanned parameters (complete with the requested scanned parameters)
        df_sim["qx"] = dic_child_collider["config_knobs_and_tuning"]["qx"]["lhcb1"]
        df_sim["qy"] = dic_child_collider["config_knobs_and_tuning"]["qy"]["lhcb1"]
        df_sim["dqx"] = dic_child_collider["config_knobs_and_tuning"]["dqx"]["lhcb1"]
        df_sim["dqy"] = dic_child_collider["config_knobs_and_tuning"]["dqy"]["lhcb1"]
        df_sim["i_bunch_b1"] = dic_child_collider["config_beambeam"]["mask_with_filling_pattern"][
            "i_bunch_b1"
        ]
        df_sim["i_bunch_b2"] = dic_child_collider["config_beambeam"]["mask_with_filling_pattern"][
            "i_bunch_b2"
        ]
        df_sim["num_particles_per_bunch"] = dic_child_collider["config_beambeam"][
            "num_particles_per_bunch"
        ]

        # Merge with particle data
        df_sim_with_particle = pd.merge(df_sim, particle, on=["particle_id"])
        l_df_to_merge.append(df_sim_with_particle)


In [20]:
node.parameters

{'config_mad': {'beam_config': {'lhcb1': {'beam_energy_tot': 6800},
   'lhcb2': {'beam_energy_tot': 6800}},
  'links': {'acc-models-lhc': '/afs/cern.ch/eng/lhc/optics/runIII'},
  'optics_file': 'acc-models-lhc/RunIII_dev/Proton_2024/V0/opticsfile.40',
  'ver_hllhc_optics': None,
  'ver_lhc_run': 3.0},
 'config_particles': {'n_angles': 5,
  'n_r': 256,
  'n_split': 5,
  'r_max': 10,
  'r_min': 2}}

Merge outputs into dataframe (but don't save yet)

In [25]:
# Merge the dataframes from all simulations together
df_all_sim = pd.concat(l_df_to_merge)



# Extract the particles that were lost for DA computation
df_lost_particles = df_all_sim[df_all_sim["state"] != 1]  # Lost particles

# Check if the dataframe is empty
if df_lost_particles.empty:
    print("No unstable particles found, the output dataframe will be empty.")

# Group by working point (Update this with the knobs you want to group by !)
group_by_parameters = ["name base collider", "qx", "qy"]
# We always want to keep beam in the final result
group_by_parameters = ["beam"] + group_by_parameters
l_parameters_to_keep = [
    "normalized amplitude in xy-plane",
    "qx",
    "qy",
    "dqx",
    "dqy",
    "i_bunch_b1",
    "i_bunch_b2",
    "num_particles_per_bunch",
]

Save as before, but call da2

In [26]:
# Min is computed in the groupby function, but values should be identical
my_final = pd.DataFrame(
    [
        df_lost_particles.groupby(group_by_parameters)[parameter].min()
        for parameter in l_parameters_to_keep
    ]
).transpose()

# Save data and print time
my_final.to_parquet(f"scans/{study_name}/da2.parquet")
print("Final dataframe for current set of simulations: ", my_final)
end = time.time()
print("Elapsed time: ", end - start)

Final dataframe for current set of simulations:                                          normalized amplitude in xy-plane  \
beam  name base collider qx     qy                                         
lhcb1 base_collider      62.305 60.309                              3.75   

                                            qx      qy   dqx   dqy  \
beam  name base collider qx     qy                                   
lhcb1 base_collider      62.305 60.309  62.305  60.309  15.0  15.0   

                                        i_bunch_b1  i_bunch_b2  \
beam  name base collider qx     qy                               
lhcb1 base_collider      62.305 60.309       411.0       410.0   

                                        num_particles_per_bunch  
beam  name base collider qx     qy                               
lhcb1 base_collider      62.305 60.309             1.150000e+11  
Elapsed time:  2912.177021741867


In [32]:
df_all_sim.to_parquet(f"scans/{study_name}/all_sim.parquet")

In [30]:
df_lost_particles.columns

Index(['start_tracking_at_element', 'q0', 'mass0', 's', 'zeta', 'chi',
       'charge_ratio', 'weight', 'particle_id', 'at_element', 'at_turn',
       'state', 'parent_particle_id', '_rng_s1', '_rng_s2', '_rng_s3',
       '_rng_s4', 'x', 'y', 'px', 'py', 'delta', 'ptau', 'rvv', 'rpp', 'p0c',
       'beta0', 'gamma0', 'path base collider', 'name base collider',
       'path simulation', 'name simulation', 'beam', 'qx', 'qy', 'dqx', 'dqy',
       'i_bunch_b1', 'i_bunch_b2', 'num_particles_per_bunch',
       'normalized amplitude in xy-plane', 'angle in xy-plane [deg]'],
      dtype='object')

In [31]:
# Min is computed in the groupby function, but values should be identical
my_full = pd.DataFrame(
    [
        df_lost_particles.groupby(group_by_parameters)[parameter].min()
        for parameter in df_lost_particles.columns
    ]
).transpose()

# Save data and print time
my_full.to_parquet(f"scans/{study_name}/da3.parquet")
print("Final dataframe for current set of simulations: ", my_final)
end = time.time()
print("Elapsed time: ", end - start)

Final dataframe for current set of simulations:                                          normalized amplitude in xy-plane  \
beam  name base collider qx     qy                                         
lhcb1 base_collider      62.305 60.309                              3.75   

                                            qx      qy   dqx   dqy  \
beam  name base collider qx     qy                                   
lhcb1 base_collider      62.305 60.309  62.305  60.309  15.0  15.0   

                                        i_bunch_b1  i_bunch_b2  \
beam  name base collider qx     qy                               
lhcb1 base_collider      62.305 60.309       411.0       410.0   

                                        num_particles_per_bunch  
beam  name base collider qx     qy                               
lhcb1 base_collider      62.305 60.309             1.150000e+11  
Elapsed time:  6643.716687679291


In [ ]:
(f"scans/{study_name}/particles.parquet")